In [26]:
""" Example for MOGONET classification
"""
from train_test import train_test

if __name__ == "__main__":    
    data_folder ='data(li)delete10//训练'
    view_list = [1,2,3]
    num_epoch_pretrain = 500
    num_epoch = 2500
    lr_e_pretrain = 1e-3
    lr_e = 5e-4
    lr_c = 1e-3
    
    if data_folder == 'ROSMAP':
        num_class = 2
    if data_folder == 'data(li)delete10//训练':
        num_class = 2
    
    train_test(data_folder, view_list, num_class,
               lr_e_pretrain, lr_e, lr_c, 
               num_epoch_pretrain, num_epoch)    


Pretrain GCNs...

Training...

Test: Epoch 0
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.583


Test: Epoch 50
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 100
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 150
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 200
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 250
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 300
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 350
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 400
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 450
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 500
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 550
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 600
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 650
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.250


Test: Epoch 700
Test ACC: 0.750
Test F1: 0.000
Test AUC: 0.25

In [27]:
###外部验证
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, average_precision_score, precision_score
from utils import gen_adj_mat_tensor, cal_adj_mat_parameter  # 确保导入你需要的函数

cuda = True if torch.cuda.is_available() else False

def load_model(model_save_path):
    # 加载最佳模型
    model_dict = torch.load(os.path.join(model_save_path, 'trained_model.model'))
    return model_dict

def preprocess_external_data(data_folder, view_list):
    # 读取外部验证集数据
    labels_ext = pd.read_csv(os.path.join(data_folder, "labels_ext.csv")).values
#    labels_ext = np.loadtxt(os.path.join(data_folder, "labels_ext.csv"), delimiter=',')
    labels_ext = labels_ext.astype(int)
    data_ext_list = []
    for i in view_list:
        data_ext_list.append(np.loadtxt(os.path.join(data_folder, f"{i}_ext.csv"), delimiter=','))
    
    data_tensor_list = []
    for i in range(len(data_ext_list)):
        data_tensor_list.append(torch.FloatTensor(data_ext_list[i]))
        if cuda:
            data_tensor_list[i] = data_tensor_list[i].cuda()
    
    return data_tensor_list, labels_ext

def generate_adj_matrix(data_ext_list, adj_parameter):
    adj_metric = "cosine"  # cosine distance
    adj_ext_list = []
    for i in range(len(data_ext_list)):
        adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, data_ext_list[i], adj_metric)
        adj_ext_list.append(gen_adj_mat_tensor(data_ext_list[i], adj_parameter_adaptive, adj_metric))
    
    return adj_ext_list

def predict_external_data(model_dict, data_ext_list, adj_ext_list):
    for m in model_dict:
        model_dict[m].eval()
    
    num_view = len(data_ext_list)
    ci_list = []
    for i in range(num_view):
        ci_list.append(model_dict[f"C{i+1}"](model_dict[f"E{i+1}"](data_ext_list[i], adj_ext_list[i])))
    
    if num_view >= 2:
        c = model_dict["C"](ci_list)
    else:
        c = ci_list[0]
    
    prob = F.softmax(c, dim=1).data.cpu().numpy()
    return prob

def calculate_metrics(true_labels, predicted_probs):
    pred_classes = np.argmax(predicted_probs, axis=1)
    AUROC = roc_auc_score(true_labels, predicted_probs, multi_class='ovr')
    AUPRC = average_precision_score(true_labels, predicted_probs, average='macro')
    accuracy = accuracy_score(true_labels, pred_classes)
    precision = precision_score(true_labels, pred_classes, average='macro')
    recall = recall_score(true_labels, pred_classes, average='macro')
    f1 = f1_score(true_labels, pred_classes, average='macro')
    return AUROC, AUPRC, accuracy, precision, recall, f1


In [28]:
# lilili
# 主程序
if __name__ == "__main__":
    model_save_path = "data(li)delete10\\训练"  # 模型保存路径
    external_data_folder = "./data(li)delete10/外部测试"  # 外部验证集路径
    view_list = [1, 2,3]  # 根据你的数据集调整视图数量
    adj_parameter = 10  # 根据你的模型设置调整邻接矩阵参数

    # 加载模型
    model_dict = load_model(model_save_path)

    # 预处理外部验证集
    data_ext_list, true_labels_ext = preprocess_external_data(external_data_folder, view_list)

    # 生成邻接矩阵
    adj_ext_list = generate_adj_matrix(data_ext_list, adj_parameter)

    # 进行预测
    predicted_probs = predict_external_data(model_dict, data_ext_list, adj_ext_list)
    print("True labels length:", len(true_labels_ext))
    print("Predicted probabilities length:", len(predicted_probs))
    print("True labels:", true_labels_ext)
    print("Predicted probabilities:", predicted_probs)
    
    # 保存预测概率和预测类别到 CSV 文件
    pred_classes = np.argmax(predicted_probs, axis=1)  # 预测的类别
    df_labels = pd.DataFrame(pred_classes, columns=['Predicted Label'])
    df_prob = pd.DataFrame(predicted_probs, columns=[f'Prob_Class_{i}' for i in range(predicted_probs.shape[1])])
    df_combined = pd.concat([df_labels, df_prob], axis=1)

    # 保存到 CSV 文件
    output_csv_path = os.path.join( 'data(li)delete10/外部测试/external_validation_predictions.csv')
    df_combined.to_csv(output_csv_path, index=False)
    print(f"Predicted labels and probabilities saved to {output_csv_path}")

C:\Users\320\AppData\Local\Temp\ipykernel_38192\1260644937.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(os.path.join(model_save_path, 'traine

True labels length: 15
Predicted probabilities length: 16
True labels: [[0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]]
Predicted probabilities: [[9.9970442e-01 2.9552967e-04]
 [9.9970442e-01 2.9552967e-04]
 [9.9970442e-01 2.9552967e-04]
 [9.9970442e-01 2.9552967e-04]
 [9.9970442e-01 2.9552967e-04]
 [9.1701847e-01 8.2981519e-02]
 [9.9970442e-01 2.9552967e-04]
 [8.5528094e-01 1.4471914e-01]
 [9.9932575e-01 6.7427114e-04]
 [9.9966633e-01 3.3363036e-04]
 [9.9970442e-01 2.9552967e-04]
 [9.9970442e-01 2.9552967e-04]
 [8.5443020e-01 1.4556974e-01]
 [8.5528094e-01 1.4471914e-01]
 [8.5528874e-01 1.4471120e-01]
 [9.9970442e-01 2.9553106e-04]]
Predicted labels and probabilities saved to data(li)delete10/外部测试/external_validation_predictions.csv


In [29]:
# 主程序
if __name__ == "__main__":
    model_save_path = "data(li)\\训练"  # 模型保存路径
    external_data_folder = "./data(li)/外部测试"  # 外部验证集路径
    view_list = [1, 2,3]  # 根据你的数据集调整视图数量
    adj_parameter = 10  # 根据你的模型设置调整邻接矩阵参数

    # 加载模型
    model_dict = load_model(model_save_path)

    # 预处理外部验证集
    data_ext_list, true_labels_ext = preprocess_external_data(external_data_folder, view_list)

    # 生成邻接矩阵
    adj_ext_list = generate_adj_matrix(data_ext_list, adj_parameter)

    # 进行预测
    predicted_probs = predict_external_data(model_dict, data_ext_list, adj_ext_list)
    print("True labels length:", len(true_labels_ext))
    print("Predicted probabilities length:", len(predicted_probs))
    print("True labels:", true_labels_ext)
    print("Predicted probabilities:", predicted_probs)
    # 计算指标
    AUROC, AUPRC, accuracy, precision, recall, f1 = calculate_metrics(true_labels_ext, predicted_probs)

    # 打印结果
    print("AUROC: {:.3f}".format(AUROC))
    print("AUPRC: {:.3f}".format(AUPRC))
    print("Accuracy: {:.3f}".format(accuracy))
    print("Precision: {:.3f}".format(precision))
    print("Recall: {:.3f}".format(recall))
    print("F1 Score: {:.3f}".format(f1))

    # 保存预测概率和预测类别到 CSV 文件
    pred_classes = np.argmax(predicted_probs, axis=1)  # 预测的类别
    df_labels = pd.DataFrame(pred_classes, columns=['Predicted Label'])
    df_prob = pd.DataFrame(predicted_probs, columns=[f'Prob_Class_{i}' for i in range(predicted_probs.shape[1])])
    df_combined = pd.concat([df_labels, df_prob], axis=1)

    # 保存到 CSV 文件
    output_csv_path = os.path.join( 'data(li)/外部测试/external_validation_predictions.csv')
    df_combined.to_csv(output_csv_path, index=False)
    print(f"Predicted labels and probabilities saved to {output_csv_path}")

True labels length: 26
Predicted probabilities length: 26
True labels: [[0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]]
Predicted probabilities: [[8.9930469e-01 1.0069536e-01]
 [8.5073966e-01 1.4926034e-01]
 [8.4834993e-01 1.5165001e-01]
 [7.6481342e-02 9.2351866e-01]
 [8.5072076e-01 1.4927919e-01]
 [8.5074055e-01 1.4925949e-01]
 [7.6587826e-02 9.2341220e-01]
 [1.9654307e-01 8.0345696e-01]
 [9.9748874e-01 2.5112869e-03]
 [9.2785216e-05 9.9990726e-01]
 [9.9748862e-01 2.5114303e-03]
 [9.9748874e-01 2.5112869e-03]
 [7.7775729e-01 2.2224267e-01]
 [8.5053712e-01 1.4946289e-01]
 [9.9244916e-01 7.5508063e-03]
 [8.2343280e-01 1.7656718e-01]
 [7.3870711e-02 9.2612934e-01]
 [7.6502360e-02 9.2349762e-01]
 [9.9748838e-01 2.5115863e-03]
 [8.4691119e-01 1.5308881e-01]
 [7.9485834e-01 2.0514166e-01]
 [8.5074079e-01 1.4925924e-01]
 [8.5065478e-01 1.4934522e-01]
 [8.4927219e-01 1.5072782e-01]
 [8.5074079e-01 1.4925919e-01

C:\Users\320\AppData\Local\Temp\ipykernel_38192\1260644937.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(os.path.join(model_save_path, 'traine

ValueError: y should be a 1d array, got an array of shape (26, 2) instead.